In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
    eloss += tf.reduce_mean(targetQs) # minimize Q
    aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
    eloss += tf.reduce_mean(eQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:0.7002 eloss:4.3245 aloss2:2.1319 exploreP:0.9980
Episode:1 meanR:19.0000 R:18.0000 rate:0.0360 aloss:0.6982 eloss:4.2298 aloss2:2.2813 exploreP:0.9962
Episode:2 meanR:21.3333 R:26.0000 rate:0.0520 aloss:0.6998 eloss:4.2404 aloss2:2.2709 exploreP:0.9937
Episode:3 meanR:20.0000 R:16.0000 rate:0.0320 aloss:0.6963 eloss:4.2152 aloss2:2.3142 exploreP:0.9921
Episode:4 meanR:20.2000 R:21.0000 rate:0.0420 aloss:0.6949 eloss:4.1910 aloss2:2.3700 exploreP:0.9901
Episode:5 meanR:18.8333 R:12.0000 rate:0.0240 aloss:0.6974 eloss:4.1858 aloss2:2.3647 exploreP:0.9889
Episode:6 meanR:17.8571 R:12.0000 rate:0.0240 aloss:0.6924 eloss:4.1757 aloss2:2.3931 exploreP:0.9877
Episode:7 meanR:17.8750 R:18.0000 rate:0.0360 aloss:0.6935 eloss:4.1896 aloss2:2.3623 exploreP:0.9859
Episode:8 meanR:17.3333 R:13.0000 rate:0.0260 aloss:0.6978 eloss:4.1411 aloss2:2.4381 exploreP:0.9847
Episode:9 meanR:19.1000 R:35.0000 rate:0.0700 aloss:0.6937 eloss:4.1232 aloss2:2.5

Episode:80 meanR:25.0247 R:20.0000 rate:0.0400 aloss:0.6837 eloss:3.8232 aloss2:3.9466 exploreP:0.8184
Episode:81 meanR:25.3659 R:53.0000 rate:0.1060 aloss:0.6788 eloss:3.7743 aloss2:3.7820 exploreP:0.8141
Episode:82 meanR:25.1807 R:10.0000 rate:0.0200 aloss:0.6826 eloss:3.7672 aloss2:3.9091 exploreP:0.8133
Episode:83 meanR:25.1071 R:19.0000 rate:0.0380 aloss:0.6777 eloss:3.7668 aloss2:3.8231 exploreP:0.8118
Episode:84 meanR:25.0000 R:16.0000 rate:0.0320 aloss:0.7000 eloss:3.7956 aloss2:3.8544 exploreP:0.8105
Episode:85 meanR:25.1860 R:41.0000 rate:0.0820 aloss:0.6838 eloss:3.7731 aloss2:3.8896 exploreP:0.8072
Episode:86 meanR:25.1724 R:24.0000 rate:0.0480 aloss:0.6992 eloss:3.8384 aloss2:3.9398 exploreP:0.8053
Episode:87 meanR:25.0000 R:10.0000 rate:0.0200 aloss:0.7007 eloss:3.7668 aloss2:3.8308 exploreP:0.8045
Episode:88 meanR:25.0112 R:26.0000 rate:0.0520 aloss:0.6868 eloss:3.7648 aloss2:3.8131 exploreP:0.8024
Episode:89 meanR:25.1111 R:34.0000 rate:0.0680 aloss:0.6934 eloss:3.7681 

Episode:159 meanR:37.9800 R:14.0000 rate:0.0280 aloss:0.7729 eloss:3.7703 aloss2:3.8739 exploreP:0.5963
Episode:160 meanR:38.8800 R:106.0000 rate:0.2120 aloss:0.7243 eloss:3.7439 aloss2:3.9212 exploreP:0.5901
Episode:161 meanR:38.9200 R:37.0000 rate:0.0740 aloss:0.7172 eloss:3.7370 aloss2:3.9527 exploreP:0.5880
Episode:162 meanR:39.0900 R:44.0000 rate:0.0880 aloss:0.7423 eloss:3.7403 aloss2:3.9642 exploreP:0.5854
Episode:163 meanR:39.2500 R:38.0000 rate:0.0760 aloss:0.7279 eloss:3.7414 aloss2:3.9404 exploreP:0.5832
Episode:164 meanR:39.3400 R:28.0000 rate:0.0560 aloss:0.6991 eloss:3.7414 aloss2:3.9478 exploreP:0.5816
Episode:165 meanR:39.2600 R:33.0000 rate:0.0660 aloss:0.7208 eloss:3.7373 aloss2:3.9396 exploreP:0.5798
Episode:166 meanR:39.6100 R:50.0000 rate:0.1000 aloss:0.7398 eloss:3.7446 aloss2:3.9521 exploreP:0.5769
Episode:167 meanR:39.8200 R:35.0000 rate:0.0700 aloss:0.7164 eloss:3.7612 aloss2:3.9364 exploreP:0.5749
Episode:168 meanR:40.0300 R:49.0000 rate:0.0980 aloss:0.7076 el

Episode:238 meanR:40.8900 R:39.0000 rate:0.0780 aloss:0.7435 eloss:3.7641 aloss2:3.9416 exploreP:0.4359
Episode:239 meanR:40.4400 R:65.0000 rate:0.1300 aloss:0.7320 eloss:3.7555 aloss2:3.9473 exploreP:0.4331
Episode:240 meanR:40.7400 R:42.0000 rate:0.0840 aloss:0.7305 eloss:3.7514 aloss2:3.9580 exploreP:0.4314
Episode:241 meanR:40.4200 R:45.0000 rate:0.0900 aloss:0.7241 eloss:3.7444 aloss2:3.9755 exploreP:0.4295
Episode:242 meanR:40.6800 R:73.0000 rate:0.1460 aloss:0.7331 eloss:3.7514 aloss2:3.9642 exploreP:0.4264
Episode:243 meanR:40.6600 R:28.0000 rate:0.0560 aloss:0.7082 eloss:3.7355 aloss2:3.9728 exploreP:0.4253
Episode:244 meanR:40.6300 R:50.0000 rate:0.1000 aloss:0.7334 eloss:3.7682 aloss2:3.9627 exploreP:0.4232
Episode:245 meanR:40.8200 R:44.0000 rate:0.0880 aloss:0.7559 eloss:3.7448 aloss2:3.9621 exploreP:0.4214
Episode:246 meanR:41.0400 R:47.0000 rate:0.0940 aloss:0.7589 eloss:3.7392 aloss2:3.9771 exploreP:0.4194
Episode:247 meanR:40.7900 R:39.0000 rate:0.0780 aloss:0.7605 elo

Episode:317 meanR:43.1800 R:36.0000 rate:0.0720 aloss:0.7955 eloss:3.7585 aloss2:3.9370 exploreP:0.3102
Episode:318 meanR:43.1900 R:33.0000 rate:0.0660 aloss:0.7571 eloss:3.7619 aloss2:3.9296 exploreP:0.3093
Episode:319 meanR:43.4600 R:54.0000 rate:0.1080 aloss:0.7403 eloss:3.7487 aloss2:3.9465 exploreP:0.3076
Episode:320 meanR:43.4100 R:45.0000 rate:0.0900 aloss:0.7599 eloss:3.7552 aloss2:3.9338 exploreP:0.3063
Episode:321 meanR:43.5100 R:46.0000 rate:0.0920 aloss:0.7729 eloss:3.7568 aloss2:3.9562 exploreP:0.3049
Episode:322 meanR:43.8500 R:54.0000 rate:0.1080 aloss:0.7368 eloss:3.7465 aloss2:3.9325 exploreP:0.3034
Episode:323 meanR:43.7200 R:34.0000 rate:0.0680 aloss:0.7443 eloss:3.7416 aloss2:3.9489 exploreP:0.3024
Episode:324 meanR:43.5300 R:32.0000 rate:0.0640 aloss:0.7349 eloss:3.7466 aloss2:3.9504 exploreP:0.3014
Episode:325 meanR:43.9200 R:80.0000 rate:0.1600 aloss:0.7405 eloss:3.7487 aloss2:3.9468 exploreP:0.2991
Episode:326 meanR:43.7200 R:38.0000 rate:0.0760 aloss:0.7463 elo

...

Episode:3632 meanR:33.6800 R:24.0000 rate:0.0480 aloss:0.4981 eloss:-19.6280 aloss2:47.0885 exploreP:0.0100
Episode:3633 meanR:33.3800 R:37.0000 rate:0.0740 aloss:0.4084 eloss:-20.2604 aloss2:45.1432 exploreP:0.0100
Episode:3634 meanR:33.8000 R:88.0000 rate:0.1760 aloss:0.4315 eloss:-19.8394 aloss2:46.4675 exploreP:0.0100
Episode:3635 meanR:33.5300 R:24.0000 rate:0.0480 aloss:0.3640 eloss:-19.9251 aloss2:47.2991 exploreP:0.0100
Episode:3636 meanR:33.2200 R:18.0000 rate:0.0360 aloss:0.4014 eloss:-24.6310 aloss2:50.5838 exploreP:0.0100
Episode:3637 meanR:33.2400 R:33.0000 rate:0.0660 aloss:0.4420 eloss:-21.0042 aloss2:47.3801 exploreP:0.0100
Episode:3638 meanR:33.0800 R:26.0000 rate:0.0520 aloss:0.4666 eloss:-18.4728 aloss2:45.2327 exploreP:0.0100
Episode:3639 meanR:32.9500 R:22.0000 rate:0.0440 aloss:0.3944 eloss:-20.2591 aloss2:45.3652 exploreP:0.0100
Episode:3640 meanR:33.3700 R:80.0000 rate:0.1600 aloss:0.4335 eloss:-20.4069 aloss2:46.0617 exploreP:0.0100
Episode:3641 meanR:33.0700 R

Episode:3708 meanR:31.5300 R:27.0000 rate:0.0540 aloss:0.3879 eloss:-18.0759 aloss2:41.1063 exploreP:0.0100
Episode:3709 meanR:31.5300 R:27.0000 rate:0.0540 aloss:0.4452 eloss:-18.8811 aloss2:41.8811 exploreP:0.0100
Episode:3710 meanR:31.5000 R:20.0000 rate:0.0400 aloss:0.4036 eloss:-21.6724 aloss2:44.5822 exploreP:0.0100
Episode:3711 meanR:31.4700 R:21.0000 rate:0.0420 aloss:0.5280 eloss:-17.4791 aloss2:40.2277 exploreP:0.0100
Episode:3712 meanR:31.4400 R:29.0000 rate:0.0580 aloss:0.4467 eloss:-18.4268 aloss2:42.4765 exploreP:0.0100
Episode:3713 meanR:31.4400 R:22.0000 rate:0.0440 aloss:0.4829 eloss:-20.0975 aloss2:43.1790 exploreP:0.0100
Episode:3714 meanR:30.8900 R:39.0000 rate:0.0780 aloss:0.4869 eloss:-18.3574 aloss2:42.0545 exploreP:0.0100
Episode:3715 meanR:30.9000 R:30.0000 rate:0.0600 aloss:0.4511 eloss:-16.2964 aloss2:38.9691 exploreP:0.0100
Episode:3716 meanR:30.7900 R:16.0000 rate:0.0320 aloss:0.4528 eloss:-20.7907 aloss2:43.6986 exploreP:0.0100
Episode:3717 meanR:30.8500 R

Episode:3784 meanR:35.6900 R:100.0000 rate:0.2000 aloss:0.3272 eloss:-19.9901 aloss2:102.0957 exploreP:0.0100
Episode:3785 meanR:35.9900 R:71.0000 rate:0.1420 aloss:0.3319 eloss:1.3902 aloss2:124.1713 exploreP:0.0100
Episode:3786 meanR:36.0700 R:36.0000 rate:0.0720 aloss:0.3460 eloss:5.6071 aloss2:134.0426 exploreP:0.0100
Episode:3787 meanR:36.1600 R:30.0000 rate:0.0600 aloss:0.3402 eloss:74.5800 aloss2:194.9779 exploreP:0.0100
Episode:3788 meanR:36.2400 R:31.0000 rate:0.0620 aloss:0.3346 eloss:-24.0012 aloss2:92.8910 exploreP:0.0100
Episode:3789 meanR:36.9600 R:98.0000 rate:0.1960 aloss:0.3209 eloss:81.6146 aloss2:170.0304 exploreP:0.0100
Episode:3790 meanR:37.0200 R:32.0000 rate:0.0640 aloss:0.3454 eloss:280.4669 aloss2:306.5457 exploreP:0.0100
Episode:3791 meanR:37.0100 R:29.0000 rate:0.0580 aloss:0.2882 eloss:-28.7067 aloss2:95.0806 exploreP:0.0100
Episode:3792 meanR:36.9500 R:26.0000 rate:0.0520 aloss:0.4467 eloss:318.2928 aloss2:311.0895 exploreP:0.0100
Episode:3793 meanR:36.8500

Episode:3860 meanR:61.0400 R:115.0000 rate:0.2300 aloss:0.1814 eloss:45.4884 aloss2:286.5322 exploreP:0.0100
Episode:3861 meanR:60.5400 R:38.0000 rate:0.0760 aloss:0.1660 eloss:216.9345 aloss2:491.6181 exploreP:0.0100
Episode:3862 meanR:60.9400 R:61.0000 rate:0.1220 aloss:0.1776 eloss:927.1086 aloss2:1224.0026 exploreP:0.0100
Episode:3863 meanR:60.6600 R:24.0000 rate:0.0480 aloss:0.2193 eloss:124.8390 aloss2:335.9911 exploreP:0.0100
Episode:3864 meanR:60.9600 R:83.0000 rate:0.1660 aloss:0.1771 eloss:88.2107 aloss2:351.3413 exploreP:0.0100
Episode:3865 meanR:61.0200 R:27.0000 rate:0.0540 aloss:0.1690 eloss:606.4897 aloss2:866.3530 exploreP:0.0100
Episode:3866 meanR:61.0600 R:35.0000 rate:0.0700 aloss:0.2239 eloss:602.4221 aloss2:569.3840 exploreP:0.0100
Episode:3867 meanR:61.1200 R:43.0000 rate:0.0860 aloss:0.1799 eloss:87.5557 aloss2:359.2127 exploreP:0.0100
Episode:3868 meanR:61.9300 R:104.0000 rate:0.2080 aloss:0.1920 eloss:621.2460 aloss2:714.8564 exploreP:0.0100
Episode:3869 meanR:

Episode:3935 meanR:72.8900 R:51.0000 rate:0.1020 aloss:0.1667 eloss:466.0811 aloss2:533.0544 exploreP:0.0100
Episode:3936 meanR:72.9700 R:120.0000 rate:0.2400 aloss:0.1926 eloss:378.6171 aloss2:611.8202 exploreP:0.0100
Episode:3937 meanR:72.4100 R:44.0000 rate:0.0880 aloss:0.1749 eloss:51.1534 aloss2:372.6896 exploreP:0.0100
Episode:3938 meanR:71.6300 R:47.0000 rate:0.0940 aloss:0.2059 eloss:1467.3135 aloss2:1173.1282 exploreP:0.0100
Episode:3939 meanR:71.4200 R:33.0000 rate:0.0660 aloss:0.1966 eloss:1819.5999 aloss2:1292.5127 exploreP:0.0100
Episode:3940 meanR:71.5900 R:70.0000 rate:0.1400 aloss:0.1866 eloss:971.1852 aloss2:926.2173 exploreP:0.0100
Episode:3941 meanR:70.9000 R:50.0000 rate:0.1000 aloss:0.3115 eloss:2268.3293 aloss2:1686.8309 exploreP:0.0100


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
